# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 57 new papers today
          25 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/24 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2209.10558


extracting tarball to tmp_2209.10558...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.10566


/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2209.10558/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'affiliations.tex' from 'tmp_2209.10558/affiliations.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.11160


/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2209.11146/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'long_table' from 'tmp_2209.11146/long_table.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


extracting tarball to tmp_2209.11160...

 done.


/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2209.11160/sample631.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.10618-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.10618) | **Atmospheric characterization of the ultra-hot Jupiter WASP-33b:  Detection of Ti and V emission lines and retrieval of a broadened line  profile**  |
|| D. Cont, et al. -- incl., <mark>Th. Henning</mark>, <mark>P. Mollière</mark> |
|*Appeared on*| *2022-09-23*|
|*Comments*| *Accepted for publication in A&A*|
|**Abstract**| Ultra-hot Jupiters are highly irradiated gas giant exoplanets on close-in orbits around their host stars. We analyzed high-resolution spectra from CARMENES, HARPS-N, and ESPaDOnS taken over eight observation nights to study the emission spectrum of WASP-33b and draw conclusions about its atmosphere. By applying the cross-correlation technique, we detected the spectral signatures of Ti I, V I, and a tentative signal of Ti II for the first time via emission spectroscopy. These detections are an important finding because of the fundamental role of Ti- and V-bearing species in the planetary energy balance. Moreover, we assessed and confirm the presence of OH, Fe I, and Si I from previous studies. The spectral lines are all detected in emission, which unambiguously proves the presence of an inverted temperature profile in the planetary atmosphere. By performing retrievals on the emission lines of all the detected species, we determined a relatively weak atmospheric thermal inversion extending from approximately 3400 K to 4000 K. We infer a supersolar metallicity close to 1.5 dex in the planetary atmosphere, and find that its emission signature undergoes significant line broadening with a Gaussian FWHM of about 4.5 km/s. Also, we find that the atmospheric temperature profile retrieved at orbital phases far from the secondary eclipse is about 300 K to 700 K cooler than that measured close to the secondary eclipse, which is consistent with different day- and nightside temperatures. Moreover, retrievals performed on the emission lines of the individual chemical species lead to consistent results, which gives additional confidence to our retrieval method. Increasing the number of species included in the retrieval and expanding the set of retrieved atmospheric parameters will further advance our understanding of exoplanet atmospheres. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.11096-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.11096) | **Companions to Kepler giant stars: A long-period eccentric substellar  companion to KIC 3526061 and a stellar companion to HD 187878**  |
|| Marie Karjalainen, et al. -- incl., <mark>Saskia Hekker</mark> |
|*Appeared on*| *2022-09-23*|
|*Comments*| *14 pages, 11 figures. Accepted to A&A*|
|**Abstract**| Context. Our knowledge of populations and occurrence of planets orbiting evolved intermediate-mass stars is still incomplete. In 2010 we started a planet-search program among 95 giant stars observed by the Kepler mission to increase the sample of giant stars with planets and with reliable estimates of stellar masses and radii. Aims. We present the two systems KIC 3526061 and HD 187878 from our planet-search program for which we could characterise their companions. Methods. We used precise stellar radial velocity measurements taken with four different echelle spectrographs to derive an orbital solution. We used Gaia astrometric measurements to obtain the inclination of the HD 187878 system and Kepler photometric observations to estimate the stellar mass and radius. Results. We report the discovery of a sub-stellar and a stellar companion around two intermediate-mass red giant branch stars. KIC 3526061 b is most likely a brown dwarf with a minimum mass of 18.15 Jupiter masses in a long-period eccentric orbit, with the orbital period 3552 d and orbital eccentricity 0.85. It is the most evolved system found having a sub-stellar companion with such a large eccentricity and wide separation. HD 187878 B has a minimum mass of 78.4 Jupiter masses. Combining the spectroscopic orbital parameters with the astrometric proper motion anomaly we derived an orbital inclination 9.8 deg, which corresponds to the companion's mass in the stellar regime of 0.51 Sun mass. Conclusions. A sub-stellar companion of KIC 3526061 extends the sample of known red giant branch stars with sub-stellar companions on very eccentric wide orbits and might provide a probe of the dynamical evolution of such systems over time. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.10558-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.10558) | **SN 2021hpr and its two siblings in the Cepheid calibrator galaxy NGC  3147: A hierarchical BayeSN analysis of a Type Ia supernova trio, and a  Hubble constant constraint**  |
|| Sam M. Ward, et al. -- incl., <mark>Mark E. Huber</mark>, <mark>Danial Langeroodi</mark> |
|*Appeared on*| *2022-09-23*|
|*Comments*| *Submitted to MNRAS; 30 pages, 22 figures*|
|**Abstract**| To improve Type Ia supernova (SN Ia) standardisability, the consistency of distance estimates to siblings -- SNe in the same host galaxy -- should be investigated. We present Young Supernova Experiment Pan-STARRS-1 $grizy$ photometry of SN 2021hpr, the third spectroscopically confirmed SN Ia in the high-stellar-mass Cepheid-calibrator galaxy NGC 3147. We analyse NGC 3147's trio of SN Ia siblings: SNe 1997bq, 2008fv and 2021hpr, using a new version of the BayeSN model of SN Ia spectral-energy distributions, retrained simultaneously using optical-NIR $BgVrizYJH$ (0.35--1.8 $\mu$m) data. The distance estimates to each sibling are consistent, with a sample standard deviation $\lesssim$0.01 mag, much smaller than the total intrinsic scatter in the training sample: $\sigma_0\approx0.09$ mag. Fitting normal SN Ia siblings in three additional galaxies, we estimate a $\approx$90% probability that the siblings' intrinsic scatter is smaller than $\sigma_0$. We build a new hierarchical model that fits light curves of siblings in a single galaxy simultaneously; this yields more precise estimates of the common distance and the dust parameters. Fitting the trio for a common dust law shape yields $R_V=2.69\pm0.52$. Our work motivates future hierarchical modelling of more siblings, to tightly constrain their intrinsic scatter, and better understand SN-host correlations. Finally, we estimate the Hubble constant, using a Cepheid distance to NGC 3147, the siblings trio, and 109 Hubble flow ($0.01 < z_{\rm{CMB}} < 0.08$) SNe Ia; marginalising over the siblings' and population's intrinsic scatters, and the peculiar velocity dispersion, yields $H_0=77.9\pm6.5 \text{ km s}^{-1}\text{Mpc}^{-1}$. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.11050-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.11050) | **A catalog of new Blazar candidates with Open Universe by High School  students**  |
|| L. Fronte, et al. -- incl., <mark>Liceo Scientifico Statale U. Morin</mark>, <mark>Technische Universitat Muenchen</mark>, <mark>Lichtenbergstrasse 2a</mark>, <mark>D-85748 Garching bei Muenchen</mark> |
|*Appeared on*| *2022-09-23*|
|*Comments*| *Proceedings of the 12th Cosmic Ray International Symposium (CRIS 2022), 12-16 September 2022, Naples (Italy). Send correspondence to: michele.doro@unipd.it, giommipaolo@gmail.com*|
|**Abstract**| Blazars are active galactic nuclei whose ultra-relativistic jets are coaligned with the observer direction. They emit throughout the whole e.m. spectrum, from radio waves to VHE gamma rays. Not all blazars are discovered. In this work, we propose a catalog of 54 new candidates based on the association of HE gamma ray emission and radio, X-ray an optical signatures. The relevance of this work is also that it was performed by four high school students from the Liceo Scientifico Statale Ugo Morin in Venice, Italy using the open-source platform Open Universe, in collaboration with the University of Padova. The framework of the activity is the Italian MIUR PCTO programme. The success of this citizen-science experience and results are hereafter reported and discussed. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.11078-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.11078) | **Harnessing Unresolved Lensed Quasars: The Mathematical Foundation of the  Fluctuation Curves**  |
|| Satadru Bag, Wuhyun Sohn, Arman Shafieloo, <mark>Kai Liao</mark> |
|*Appeared on*| *2022-09-23*|
|*Comments*| *20 pages, 8 figures, 1 table*|
|**Abstract**| Strong gravitational lensed quasars (QSOs) have emerged as powerful and novel cosmic probes as they can deliver crucial cosmological information, such as a measurement of the Hubble constant, independent of other probes. Although the upcoming LSST survey is expected to discover $10^3-10^4$ lensed QSOs, a large fraction will remain unresolved due to seeing. The stochastic nature of the quasar intrinsic flux makes it challenging to identify lensed ones and measure the time delays using unresolved light curve data only. In this regard, Bag et al (2022) introduced a data-driven technique based on the minimization of the fluctuation in the reconstructed image light curves. In this article, we delve deeper into the mathematical foundation of this approach. We show that the lensing signal in the fluctuation curve is dominated by the auto-correlation function (ACF) of the derivative of the joint light curve. This explains why the fluctuation curve enables the detection of the lensed QSOs only using the joint light curve, without making assumptions about QSO flux variability, nor requiring any additional information. We show that the ACF of the derivative of the joint light curve is more reliable than the ACF of the joint light curve itself because intrinsic quasar flux variability shows significant auto-correlation up to a few hundred days (as they follow a red power spectrum). In addition, we show that the minimization of fluctuation approach provides even better precision and recall as compared to the ACF of the derivative of the joint light curve when the data have significant observational noise. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.11146-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.11146) | **MLGWSC-1: The first Machine Learning Gravitational-Wave Search Mock Data  Challenge**  |
|| Marlin B. Schäfer, et al. -- incl., <mark>He Wang</mark>, <mark>Zong-Kuan Guo</mark>, <mark>Frank Ohme</mark> |
|*Appeared on*| *2022-09-23*|
|*Comments*| *25 pages, 6 figures, 4 tables, additional material available at this https URL*|
|**Abstract**| We present the results of the first Machine Learning Gravitational-Wave Search Mock Data Challenge (MLGWSC-1). For this challenge, participating groups had to identify gravitational-wave signals from binary black hole mergers of increasing complexity and duration embedded in progressively more realistic noise. The final of the 4 provided datasets contained real noise from the O3a observing run and signals up to a duration of 20 seconds with the inclusion of precession effects and higher order modes. We present the average sensitivity distance and runtime for the 6 entered algorithms derived from 1 month of test data unknown to the participants prior to submission. Of these, 4 are machine learning algorithms. We find that the best machine learning based algorithms are able to achieve up to 95% of the sensitive distance of matched-filtering based production analyses for simulated Gaussian noise at a false-alarm rate (FAR) of one per month. In contrast, for real noise, the leading machine learning search achieved 70%. For higher FARs the differences in sensitive distance shrink to the point where select machine learning submissions outperform traditional search algorithms at FARs $\geq 200$ per month on some datasets. Our results show that current machine learning search algorithms may already be sensitive enough in limited parameter regions to be useful for some production settings. To improve the state-of-the-art, machine learning algorithms need to reduce the false-alarm rates at which they are capable of detecting signals and extend their validity to regions of parameter space where modeled searches are computationally expensive to run. Based on our findings we compile a list of research areas that we believe are the most important to elevate machine learning searches to an invaluable tool in gravitational-wave signal detection. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.11160-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.11160) | **TOI-5205b: A Jupiter transiting an M dwarf near the Convective Boundary**  |
|| Shubham Kanodia, et al. -- incl., <mark>Suvrath Mahadevan</mark>, <mark>Jessica Libby-Roberts</mark>, <mark>Andrea S.J. Lin</mark>, <mark>Henry A. Kobulnicky</mark> |
|*Appeared on*| *2022-09-23*|
|*Comments*| *Submitted to ApJ. Comments are welcome. arXiv admin note: text overlap with arXiv:2203.07178*|
|**Abstract**| We present the discovery of TOI-5205b, a transiting Jovian planet orbiting a solar metallicity M4V star, which was discovered using TESS photometry and then confirmed using a combination of precise radial velocities, ground-based photometry, spectra and speckle imaging. The host star TOI-5205 sits near the eponymous `Jao gap', which is the transition region between partially and fully-convective M dwarfs. TOI-5205b has one of the highest mass ratio for M dwarf planets with a mass ratio of almost 0.3$\%$, as it orbits a host star that is just $0.392 \pm 0.015$ $M_{\odot}$. Its planetary radius is $1.03 \pm 0.03~R_J$, while the mass is $1.08 \pm 0.06~M_J$. Additionally, the large size of the planet orbiting a small star results in a transit depth of $\sim 7\%$, making it one of the deepest transits of a confirmed exoplanet orbiting a main-sequence star. The large transit depth makes TOI-5205b a compelling target to probe its atmospheric properties, as a means of tracing the potential formation pathways. While there have been radial velocity-only discoveries of giant planets around mid M dwarfs, this is the first transiting Jupiter with a mass measurement discovered around such a low-mass host star. The high mass of TOI-5205b stretches conventional theories of planet formation and disk scaling relations that cannot easily recreate the conditions required to form such planets. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.10635-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.10635) | **A primordial atmospheric origin of hydrospheric deuterium enrichment on  Mars**  |
|| Kaveh Pahlevan, et al. -- incl., <mark>Linda T. Elkins-Tanton</mark> |
|*Appeared on*| *2022-09-23*|
|*Comments*| *5 figures*|
|**Abstract**| The deuterium-to-hydrogen (D/H or 2H/1H) ratio of Martian atmospheric water (~6x standard mean ocean water, SMOW) is higher than that of known sources, requiring planetary enrichment. A recent measurement by NASA's Mars Science Laboratory rover Curiosity of >3 Gyr clays yields a D/H ratio ~3x SMOW, demonstrating that most enrichment occurs early in Mars's history. As on Venus, Mars's D/H enrichment is thought to reflect preferential loss to space of 1H (protium) relative to 2H (deuterium), but the global environmental context of large and early hydrogen losses remain to be determined. Here, we apply a recent model of primordial atmosphere evolution to Mars, link the magma ocean of the accretion epoch with a subsequent water-ocean epoch, and calculate the behavior of deuterium for comparison with the observed record. We find that a ~2-3x hydrospheric deuterium-enrichment is produced if the Martian magma ocean is chemically reducing at last equilibration with the primordial atmosphere, making H2-CO the initially dominant species, with minor abundances of H2O-CO2. Reducing gases - in particular H2 - can cause greenhouse warming and prevent a water ocean from freezing immediately after the magma ocean epoch. Moreover, the pressure-temperature conditions are high enough to produce ocean-atmosphere H2O-H2 isotopic equilibrium such that surface H2O strongly concentrates deuterium relative to H2, which preferentially takes up protium and escapes from the primordial atmosphere. The proposed scenario of primordial H2-rich outgassing and escape suggests significant durations (>Myr) of chemical conditions on the Martian surface conducive to prebiotic chemistry immediately following Martian accretion. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2209.10618.md
    + _build/html/tmp_2209.10618/./Figures/sysrem.png
    + _build/html/tmp_2209.10618/./Figures/corner_plot_all.png
    + _build/html/tmp_2209.10618/./Figures/SNmaps_and_template_spectra_neutral.png
exported in  _build/html/2209.11096.md
    + _build/html/tmp_2209.11096/./fig8.png
    + _build/html/tmp_2209.11096/./fig2.png
    + _build/html/tmp_2209.11096/./fig6.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.4}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\arraystretch$}{1.3}$
$\newcommand{$\arraystretch$}{1.3}$
$\newcommand{$\arraystretch$}{1.3}$
$\newcommand{$\arraystretch$}{1.3}$
$\newcommand{$\arraystretch$}{1.4}$</div>



<div id="title">

# Atmospheric characterization of the ultra-hot Jupiter WASP-33b: Detection of Ti and V emission lines and retrieval of a broadened line profile

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2209.10618-b31b1b.svg)](https://arxiv.org/abs/2209.10618)<mark>Appeared on: 2022-09-23</mark> - _Accepted for publication in A&A_

</div>
<div id="authors">

D. Cont, et al. -- incl., <mark><mark>Th. Henning</mark></mark>, <mark><mark>P. Mollière</mark></mark>

</div>
<div id="abstract">

**Abstract:** Ultra-hot Jupiters are highly irradiated gas giant exoplanets on close-in orbits around their host stars. The dayside atmospheres of these objects strongly emit thermal radiation due to their elevated temperatures, making them prime targets for characterization by emission spectroscopy. We analyzed high-resolution spectra from CARMENES, HARPS-N, and ESPaDOnS taken over eight observation nights to study the emission spectrum of WASP-33b and draw conclusions about its atmosphere. By applying the cross-correlation technique, we detected the spectral signatures of$\ion${Ti}{i},$\ion${V}{i}, and a tentative signal of$\ion${Ti}{ii}for the first time via emission spectroscopy. These detections are an important finding because of the fundamental role of Ti- and V-bearing species in the planetary energy balance. Moreover, we assessed and confirm the presence of OH,$\ion${Fe}{i}, and$\ion${Si}{i}from previous studies. The spectral lines are all detected in emission, which unambiguously proves the presence of an inverted temperature profile in the planetary atmosphere. By performing retrievals on the emission lines of all the detected species, we determined a relatively weak atmospheric thermal inversion extending from approximately 3400 K to 4000 K. We infer a supersolar metallicity close to 1.5 dex in the planetary atmosphere, and find that its emission signature undergoes significant line broadening with a Gaussian FWHM of about 4.5 km s$^{-1}$. Also, we find that the atmospheric temperature profile retrieved at orbital phases far from the secondary eclipse is about 300 K to 700 K cooler than that measured close to the secondary eclipse, which is consistent with different day- and nightside temperatures. Moreover, retrievals performed on the emission lines of the individual chemical species lead to consistent results, which gives additional confidence to our retrieval method. Increasing the number of species included in the retrieval and expanding the set of retrieved atmospheric parameters will further advance our understanding of exoplanet atmospheres.

</div>

<div id="div_fig1">

<img src="tmp_2209.10618/./Figures/sysrem.png" alt="Fig1" width="100%"/>

**Figure 1. -** Data reduction steps for a representative CARMENES NIR wavelength range. {\it Panel a} shows the unprocessed spectral matrix. {\it Panel b} is the spectral matrix after normalization and outlier correction; we mask the strongest telluric lines in this step. {\it Panel c} shows the spectra reduced with \texttt{SYSREM} after telluric and stellar line removal. (*SYSREM*)

</div>
<div id="div_fig2">

<img src="tmp_2209.10618/./Figures/corner_plot_all.png" alt="Fig7" width="100%"/>

**Figure 7. -** Posterior distributions of the atmospheric parameters from our retrieval. We used the emission lines of all the detected species (i.e., $\ion${Ti}{i}, $\ion${V}{i}, OH, $\ion${Fe}{i}, $\ion${Si}{i}, and $\ion${Ti}{ii}). (*corner_plot_all*)

</div>
<div id="div_fig3">

<img src="tmp_2209.10618/./Figures/SNmaps_and_template_spectra_neutral.png" alt="Fig11" width="100%"/>

**Figure 11. -** Model spectra and S/N maps of the neutral chemical species investigated. The \textit{left panels} show the normalized model spectra of the detected species. The presented interval 3700--17 100 $\AA$ corresponds to the combined wavelength coverage of the instruments used. The \textit{right panels} show the S/N maps of the neutral species that we searched for. For each species, the S/N map corresponds to the specific \texttt{SYSREM} iteration that maximizes the detection. The detection peaks are indicated by the yellow dash-dotted lines. (*SNmaps_and_template_spectra_neutral*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$</div>



<div id="title">

#  giant stars: A long-period eccentricsubstellar companion to KIC 3526061 and a stellar companion to HD 187878

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2209.11096-b31b1b.svg)](https://arxiv.org/abs/2209.11096)<mark>Appeared on: 2022-09-23</mark> - _14 pages, 11 figures. Accepted to A&A_

</div>
<div id="authors">

Marie Karjalainen, et al. -- incl., <mark><mark>Saskia Hekker</mark></mark>

</div>
<div id="abstract">

**Abstract:** Our knowledge of populations and occurrence of planets orbiting evolvedintermediate-mass stars is still incomplete. In 2010 westarted a planet-search program among 95 giant starsobserved by the\textit{Kepler}mission to increase the sampleof giant stars with planets and with reliable estimates of stellar massesand radii.We present the two systems KIC 3526061 and HD 187878from our planet-search program for which we could characterise theircompanions.We used precise stellar radial velocity measurements taken with fourdifferent echelle spectrographs to derive an orbital solution.We used\textit{Gaia}astrometric measurements to obtain the inclinationof the HD 187878 system and\textit{Kepler}photometric observations to estimate the stellarmass and radius.We report the discovery of a sub-stellar and a stellar companion around twointermediate-mass red giant branch stars. KIC 3526061 b is most likely a brown dwarf with aminimum mass of$18.15\pm 0.44 {\rm{M_{Jupiter}}}$in a long-periodeccentric orbit, with the orbital period$3552_{-135}^{+158}$d andorbital eccentricity$e=0.85\pm 0.01$. It is the most evolved system foundhaving a sub-stellar companion with such a large eccentricity and wide separation.HD 187878 B has a minimum mass of$78.4\pm 2.0 {\rm{M_{Jupiter}}}$.Combining the spectroscopic orbital parameters with the astrometric propermotion anomaly we derived an orbital inclination$i=9.8^{+0.4}_{-0.6}$deg, whichcorresponds to the companion's mass in the stellar regime of$0.51^{+0.04}_{-0.02} \rm{M_{\odot}}$.A sub-stellar companion of KIC 3526061 extends the sample of known red giantbranch stars with sub-stellar companions on very eccentric wide orbits andmight provide a probe of the dynamical evolution of such systems over time.

</div>

<div id="div_fig1">

<img src="tmp_2209.11096/./fig8.png" alt="Fig7" width="100%"/>

**Figure 7. -** Frequency amplitude spectra for KIC 3526061.
Top: Bisector velocity spans. Middle: Equivalent width of the $\ion${Ca}{ii}
triplet lines. Bottom: Full-width at half maximum of spectral line shapes.
For all three quantities we found variations on a similar time scale. (*Fig_BVS_FWHM_EQW*)

</div>
<div id="div_fig2">

<img src="tmp_2209.11096/./fig2.png" alt="Fig2" width="100%"/>

**Figure 2. -** RV measurements of KIC 3526061. Top: Data obtained from
August 2012 to July 2021 using the TS2 spectrograph at the McDonald
Observatory, Texas, and the HERMES spectrograph at Mercator,
La Palma. The solid curve represents the Keplerian orbital solution.
Bottom: RV residuals and error bars
after removing the brown dwarf's orbital solution. (*Fig_kic_orbital*)

</div>
<div id="div_fig3">

<img src="tmp_2209.11096/./fig6.png" alt="Fig6" width="100%"/>

**Figure 6. -** Eccentricity versus semi-major axis for all sub-stellar companions known around
giant host stars
(source: https://www.lsw.uni-heidelberg.de/users/sreffert/giantplanets/giantplanets.php).
The red asterisk shows the position of KIC 3526061 b. (*Fig_evsa*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

42  publications in the last 7 days.
	 _build/html/2209.11096.md
	 _build/html/2209.10618.md
	 _build/html/2209.09914.md
	 _build/html/2209.09823.md
	 _build/html/2209.08242.md
	 _build/html/2209.08120.md
	 _build/html/2209.08106.md
	 _build/html/2209.07962.md
	 _build/html/2209.07931.md
	 _build/html/2209.06833.md
	 _build/html/2209.05814.md
	 _build/html/2209.05593.md
	 _build/html/2209.04304.md
	 _build/html/2209.03367.md
	 _build/html/2209.02872.md
	 _build/html/2209.02725.md
	 _build/html/2209.02722.md
	 _build/html/2209.02092.md
	 _build/html/2209.01125.md
	 _build/html/2209.00828.md
	 _build/html/2209.00363.md
	 _build/html/2208.14927.md
	 _build/html/2208.09335.md
	 _build/html/2208.08872.md
	 _build/html/2204.06393.md
	 _build/html/2204.03335.md
	 _build/html/2204.03253.md
	 _build/html/2204.02998.md
	 _build/html/2204.02109.md
	 _build/html/2204.02017.md
	 _build/html/2204.01824.md
	 _build/html/2204.01758.md
	 _build/html/2204.01245.md
	 _build/html/2204.00793.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers